<a href="https://colab.research.google.com/github/GiuliaMaragno/Progetto-Sii-ML/blob/main/ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from matplotlib import pyplot as plt




from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
     

train valid test 70 20 10


> Modello: ResNet 50

> target_size=224, 224

> batch_size=100

> optimizer: SDG

> epoche: 25


In [4]:
train_path ='/content/drive/MyDrive/Progetto/kvasir-dataset-v2/Train'
valid_path='/content/drive/MyDrive/Progetto/kvasir-dataset-v2/Valid'
test_path='/content/drive/MyDrive/Progetto/kvasir-dataset-v2/Test'

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.1, horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_set = train_datagen.flow_from_directory(directory=train_path, target_size=(224, 224), class_mode='categorical', batch_size=100)
valid_set = valid_datagen.flow_from_directory(directory=valid_path, target_size=(224, 224), class_mode='categorical', batch_size=100)
test_set = valid_datagen.flow_from_directory(directory=test_path, target_size=(224,224), class_mode='categorical', batch_size=1, shuffle=False)

Found 5646 images belonging to 8 classes.
Found 1600 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


In [34]:
print("Batch Size for Input Image train_set: ",train_set[0][0].shape)
print("Batch Size for Input Image valid_set: ",valid_set[0][0].shape)
print("Batch Size for Input Image test_set: ",test_set[0][0].shape)
print("Batch Size for Output Image train_set: ",train_set[0][1].shape)
print("Batch Size for Output Image valid_set: ",valid_set[0][1].shape)
print("Batch Size for Output Image test_set: ",test_set[0][1].shape)
print("Image Size of first image train: ",train_set[0][0][0].shape)
print("Image Size of first image valid: ",valid_set[0][0][0].shape)
print("Image Size of first image test: ",test_set[0][0][0].shape)
print("Output of first image train: ",train_set[0][1][0].shape)
print("Output of first image valid: ",valid_set[0][1][0].shape)
print("Output of first image test: ",test_set[0][1][0].shape)

KeyboardInterrupt: ignored

In [10]:
k=train_set[0][0].shape[1:]
print(k)

(512, 512, 3)


In [21]:
resNet=tf.keras.applications.ResNet50(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,

)

102967424/102967424 [==============================] - 5s 0us/step


In [22]:
x = resNet.layers[-2].output
output = Dense(units=8, activation='softmax')(x)
model = Model(inputs=resNet.input, outputs=output)

In [23]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [24]:
from tensorflow.python.keras import optimizers

sgd = tf.keras.optimizers.SGD(learning_rate=0.01, weight_decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [25]:
step_size_train=train_set.n//train_set.batch_size
step_size_valid=valid_set.n//valid_set.batch_size

In [26]:
model.fit(x=train_set, steps_per_epoch=step_size_train, validation_data= valid_set, validation_steps=step_size_valid, epochs=25)

Epoch 1/25
56/56 [==============================] - 4931s 88s/step - loss: 0.5189 - accuracy: 0.8022 - val_loss: 7.3253 - val_accuracy: 0.1231
Epoch 2/25
56/56 [==============================] - 214s 4s/step - loss: 0.2050 - accuracy: 0.9158 - val_loss: 3.3405 - val_accuracy: 0.1213
Epoch 3/25
56/56 [==============================] - 216s 4s/step - loss: 0.1548 - accuracy: 0.9407 - val_loss: 7.8438 - val_accuracy: 0.1750
Epoch 4/25
56/56 [==============================] - 211s 4s/step - loss: 0.1132 - accuracy: 0.9591 - val_loss: 6.4715 - val_accuracy: 0.1250
Epoch 5/25
56/56 [==============================] - 220s 4s/step - loss: 0.0842 - accuracy: 0.9683 - val_loss: 8.3988 - val_accuracy: 0.1200
Epoch 6/25
56/56 [==============================] - 208s 4s/step - loss: 0.0678 - accuracy: 0.9758 - val_loss: 4.7322 - val_accuracy: 0.1250
Epoch 7/25
56/56 [==============================] - 219s 4s/step - loss: 0.0422 - accuracy: 0.9858 - val_loss: 4.0577 - val_accuracy: 0.1187
Epoch 8/25


In [27]:
model.save("/content/drive/MyDrive/Progetto/Models/Modello ResNet50")

In [6]:
pred = model.predict(test_set, steps = len(test_set), verbose = 2)

800/800 - 238s - 238s/epoch - 298ms/step


In [2]:
model= keras.models.load_model("/content/drive/MyDrive/Progetto/Models/Modello ResNet50")

In [7]:
predicted_class = np.argmax(pred, axis = 1)
true_classes = test_set.classes
class_labels= list(test_set.class_indices.keys())

from sklearn.metrics import classification_report
report = classification_report(true_classes, predicted_class, target_names=class_labels)
print(report)

                        precision    recall  f1-score   support

    dyed-lifted-polyps       0.97      0.95      0.96       100
dyed-resection-margins       0.96      0.98      0.97       100
           esophagitis       0.82      0.80      0.81       100
          normal-cecum       0.96      1.00      0.98       100
        normal-pylorus       0.97      0.99      0.98       100
         normal-z-line       0.80      0.82      0.81       100
                polyps       0.94      0.96      0.95       100
    ulcerative-colitis       0.99      0.91      0.95       100

              accuracy                           0.93       800
             macro avg       0.93      0.93      0.93       800
          weighted avg       0.93      0.93      0.93       800

